In [22]:
(ns vlado)

nil


In [23]:
(def program "1 2 + 4 +")

#'vlado/program


In [24]:
(read-string (str "[" program "]"))

[1 2 + 4 +]


In [25]:
(defprotocol IEvaluate
    "protocol for evaluation of prgram"
    (-eval [this state]))

IEvaluate


In [26]:
(defrecord State [ds ps env])

vlado.State


In [63]:
(extend-protocol IEvaluate
    java.lang.Long
    (-eval [this state]
        (update-in state [:ds] conj this))
    clojure.lang.Symbol
    (-eval [this state]
        (let [[a b & ds] (:ds state)
              f (this (:env state))]
          (f state)
)))
;;(extends? java.lang.Long IEvaluate)

nil


In [28]:
(defn wrap-function [f num-args]
    (fn [{:keys [ds ps] :as state}]
        (let [args (take num-args ds)
              rest-args (drop num-args ds)]
              (assoc state :ds (vec (concat [(apply f args)] rest-args))))))

#'vlado/wrap-function


In [61]:
(defn if-function [{:keys [ds ps] :as state}]
    (let [[pred & ds] ds
          [then else & ps] ps]
        (assoc state :ps (concat ps (if pred then else))
                     :ds ds)
    )
)

#'vlado/if-function


In [68]:
(defn init-keywords []
    {'+ (wrap-function + 2)
     '- (wrap-function - 2)
     '* (wrap-function * 2)
     '= (wrap-function = 2)
     '< (wrap-function < 2)
     '> (wrap-function > 2)
     'if if-function
    }
)

#'vlado/init-keywords


In [31]:
(defn init-state [program]
    (State. [] (read-string (str "[" program "]")) (init-keywords)))

;;(init-state program)

#'vlado/init-state


In [67]:
(defn interpret [program]
    (loop [state (init-state program)]
        (let [ch (first (:ps state))
              state (update-in state [:ps] rest)]
            (if ch
                (recur (-eval ch state))
                (first (:ds state))
))))

(interpret "1 2 3 + * 9 = if [2 1 +] [2 1 -]")

#'vlado/interpret
3


In [70]:
(interpret "1 2 < if [0] [1]")

0
